# Drizzle 2: Bayer drizzle

Drizzle separately each of the R, G, and B arrays of a Bayer image. 

At the end, combine the resulting 3 arrays as a 3-extension FITS file. 

It is assujed the input was already background-subtracted.

Originally this notebook was developed with the ISO 12800 data set as the test data. In the current version, it is configured to use the ISO 6400 data set. Statements specific to the original data set were comented out.

In [1]:
# %matplotlib ipympl

import os, glob
import datetime

import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.visualization import make_lupton_rgb
from drizzle import cdrizzle
import rawpy

from datapath import DATA

# for background subtraction experiment
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans
from astropy.stats import SigmaClip
from photutils import Background2D, MedianBackground, ModeEstimatorBackground

/var/folders/y5/zcwj7zc17g19nggvb1vx4rc80000gn/T/ipykernel_10606/3982284005.py:20: DeprecationWarning: `photutils.Background2D` is a deprecated alias for `photutils.background.Background2D` and will be removed in the future. Instead, please use `from photutils.background import Background2D` to silence this warning.
  from photutils import Background2D, MedianBackground, ModeEstimatorBackground
/var/folders/y5/zcwj7zc17g19nggvb1vx4rc80000gn/T/ipykernel_10606/3982284005.py:20: DeprecationWarning: `photutils.MedianBackground` is a deprecated alias for `photutils.background.MedianBackground` and will be removed in the future. Instead, please use `from photutils.background import MedianBackground` to silence this warning.
  from photutils import Background2D, MedianBackground, ModeEstimatorBackground
/var/folders/y5/zcwj7zc17g19nggvb1vx4rc80000gn/T/ipykernel_10606/3982284005.py:20: DeprecationWarning: `photutils.ModeEstimatorBackground` is a deprecated alias for `photutils.background.ModeE

## Functions

In [2]:
def drizz_rgb(imarray, pixmap, output_array, weights, output_counts, output_context, 
              aslice=None):
    # for diagnostic purposes, this function will also return stats associated 
    # with each one of the input RGB arrays
    stats = []
    
    # loop over the RGB bands    
    for band in list(range(3)):
        
        # generate zeroed array and add the input rawpy array to it. This is necessary 
        # since calling cdrizzle.tdriz directly with the rawpy-generated array as input,
        # causes an exception to be raised by cdrizzle. 
        # 
        # We also need to apply the mask/weights here, because drizzle will re-scale 
        # the mask when doing pixel re-scaling.
        imarray_copy = np.zeros(imarray.shape, dtype='float32')
        imarray_copy += imarray * weights[band]

        # stats for diagnostics
        imarray_stats = imarray_copy
        imarray_stats[imarray_copy == 0.] = np.NaN
        stats.append(np.nanmean(imarray_stats[aslice]))
        
        # call C core drizzle algorithm
        cdrizzle.tdriz(imarray_copy, weights[band], pixmap, output_array[band], 
                       output_counts[band], output_context[band], 
                       kernel="turbo", expscale=1.)
        
    return stats

In [3]:
def drizz(filename, output_array, weights, output_counts, output_context):
    '''
    Drizzle one raw image. 
    
    This works by breaking up the raw array into each one of its 3 sub-images,
    R, G, and B, and drizzling them separately into their own respective
    output arrays. A single pixel mapping file containing the offsets associated 
    with the input raw image is used. This is the file created by the Offsets_X 
    scripts.
    
    The outputs are 3-tuples that store respectively the R, G, and B results 
    in each one of their elements. They have to be allocated and initialized
    by the caller.
    
    The weights array must have the RGB pixel masks multiplied into them.
    '''
    # read input
#     raw = rawpy.imread(filename)
#     imarray = raw.raw_image_visible.astype(float)

    f = fits.open(filename)
    imarray = f[1].data
    f.close()   
    
    # read offsets
    fname_offsets = filename.replace('cutout.fits', 'offsets.fits')
    f = fits.open(fname_offsets)
    offsets_x = f[1].data
    offsets_y = f[2].data
    f.close()
    
    # build 1-to-1 pixmap
    idxmap = np.indices((imarray.shape[1], imarray.shape[0]), dtype='float64')

    # add offsets
    idxmap[0] -= offsets_x.transpose()
    idxmap[1] -= offsets_y.transpose()

    # reshape for drizzle
    idxmap = idxmap.transpose()
    idxmap = idxmap.reshape(imarray.shape[1] * imarray.shape[0], 2)
    pixmap = idxmap.reshape(imarray.shape[0], imarray.shape[1], 2)
    
    # call drizzle on the RGB arrays
#     drizz_rgb(subtracted, pixmap, output_array, weights, output_counts, output_context)
    stats = drizz_rgb(imarray, pixmap, output_array, weights, output_counts, output_context,
                      aslice=np.index_exp[700:800,1250:1300])
    
    # print stats
    _, fname = os.path.split(fpath)
    print(fname, stats)

## Read reference image

In [4]:
# reference image used in the Offsets_X scripts. Offsets are defined against this image.
# This reference is used for both the ISO 12800 and 6400 data sets.
# datadir = os.path.join(DATA,'astrophotography_data/MilkyWayPrettyBoy/12800/light')
# reference_fname = os.path.join(datadir,'DSC03770.ARW')

# reference for the Andromeda_2022 data set
datadir = os.path.join(DATA,'astrophotography_data/Andromeda_2022/135mm1s3200ISO/')
reference_fname = os.path.join(datadir,'DSC00362.cutout.fits')
reference_fname_arw = os.path.join(datadir,'DSC00362.ARW')

f = fits.open(reference_fname)
reference_imarray = f[1].data
f.close()    

raw = rawpy.imread(reference_fname_arw)

## Build masks that isolate the R, G, and B layers from a Bayer array

This step must run only after an actual image was read by rawpy, so the Bayer encoding of said image becomes available.

In [5]:
# Bayer code mask should be the same size as cutout
yr0 = 0
yr1 = reference_imarray.shape[0]
xr0 = 0
xr1 = reference_imarray.shape[1]

bayer_slice = np.index_exp[yr0:yr1,xr0:xr1]

In [6]:
colors_array = raw.raw_colors_visible
# mask = np.ones(shape=colors_array.shape, dtype='float32')

red_mask = np.where(colors_array == 0, 1, 0).astype(np.float32)

green_mask_1 = np.where(colors_array == 1, 1, 0)
green_mask_2 = np.where(colors_array == 3, 1, 0)
green_mask = (green_mask_1 | green_mask_2).astype(np.float32)

blue_mask = np.where(colors_array == 2, 1, 0).astype(np.float32)

masks = []
masks.append(red_mask[bayer_slice])
masks.append(green_mask[bayer_slice])
masks.append(blue_mask[bayer_slice])

## Parameters to control background subtraction

In [7]:
bkg_cell_footprint = (25, 25)
bkg_filter = (9, 9)

bkg_sigma_clip = SigmaClip(sigma=5.)
bkg_kernel = Gaussian2DKernel(x_stddev=3)
bkg_estimator = ModeEstimatorBackground()

## Define list of images to be processed

In [8]:
# path1 = os.path.join(DATA,'astrophotography_data/MilkyWayPrettyBoy/12800/light')   # ISO 12800 data set
# path2 = os.path.join(DATA,'astrophotography_data/MilkyWayPrettyBoy/6400/light')   # ISO 6400 data set

# list_p_1 = glob.glob(path1 + '/*.ARW')[1:]
# list_p_2 = glob.glob(path2 + '/*.ARW')
# list_p = list_p_2[20:] + list_p_1

# list_r = [reference_fname]

# list_p.sort()
# list_p.reverse()
# list_p = list_r + list_p


# Andromeda_2022 data set
list_p = glob.glob(datadir + '/*.cutout.fits')
list_p.sort()
list_p = list_p[1:]

## Allocate and initialize arrays used by drizzle

Use Bayer masks as weights for drizzling.

In [9]:
weights = []
output_array = []
output_counts = []
output_context = []

for band in list(range(3)):

    weights.append(np.ones(reference_imarray.shape, dtype='float32') * masks[band])

    output_array.append(np.zeros(reference_imarray.shape, dtype='float32'))
    output_counts.append(np.zeros(reference_imarray.shape, dtype='float32'))
    output_context.append(np.zeros(reference_imarray.shape, dtype='int32'))

In [10]:
# print(weights[0])
# print()
# print(weights[1])
# print()
# print(weights[2])

## Loop over list of images

In [ ]:
for fpath in list_p:
    drizz(fpath, output_array, weights, output_counts, output_context)

DSC00363.cutout.fits [2.0793877, 5.0065556, 2.6852643]
DSC00364.cutout.fits [2.6665218, 5.63182, 2.5881796]
DSC00365.cutout.fits [2.2247207, 4.454556, 2.2511058]
DSC00366.cutout.fits [2.469363, 5.270381, 3.0612743]
DSC00367.cutout.fits [1.9781711, 5.626021, 2.6189024]
DSC00368.cutout.fits [2.1710935, 5.0270834, 2.7624366]
DSC00369.cutout.fits [1.4701759, 4.254408, 3.1372519]
DSC00370.cutout.fits [2.5966477, 4.737225, 4.165671]
DSC00371.cutout.fits [1.6920176, 5.4810476, 3.4527345]
DSC00372.cutout.fits [3.3131456, 5.9120846, 2.3844695]
DSC00373.cutout.fits [1.0639819, 4.770171, 2.3355894]
DSC00374.cutout.fits [1.8909637, 4.053674, 2.8743298]
DSC00375.cutout.fits [2.086118, 5.1310353, 1.667727]
DSC00376.cutout.fits [2.2968366, 4.9438314, 2.4556499]
DSC00377.cutout.fits [1.7751939, 4.8697433, 3.989516]
DSC00378.cutout.fits [3.2441812, 4.5908813, 2.4360156]
DSC00379.cutout.fits [2.8756757, 5.515987, 3.1533036]
DSC00380.cutout.fits [1.1957779, 5.290493, 3.3220868]
DSC00381.cutout.fits [2.67

DSC00515.cutout.fits [3.0817142, 2.5686195, 2.411782]
DSC00516.cutout.fits [1.797348, 2.7095695, 4.6127915]
DSC00517.cutout.fits [2.533425, 2.7983632, 3.397384]
DSC00518.cutout.fits [1.5410666, 2.2008348, 3.6480196]
DSC00519.cutout.fits [1.4450287, 2.1339788, 3.049027]
DSC00520.cutout.fits [1.993382, 2.3548598, 3.7405899]
DSC00521.cutout.fits [1.2400773, 2.6734083, 3.0078475]
DSC00522.cutout.fits [2.6876066, 3.2652755, 4.2871923]
DSC00523.cutout.fits [3.2511547, 2.8058534, 3.330573]
DSC00524.cutout.fits [2.3464837, 2.7273574, 3.8249664]
DSC00525.cutout.fits [1.4204872, 2.7371516, 3.4559758]
DSC00526.cutout.fits [1.802839, 2.279517, 3.7757328]
DSC00527.cutout.fits [2.2951753, 2.8240798, 3.337203]
DSC00528.cutout.fits [1.6695582, 2.7545798, 3.8882484]
DSC00529.cutout.fits [2.6414132, 3.4423625, 3.2119806]
DSC00530.cutout.fits [2.4797215, 2.6250076, 2.9855871]
DSC00531.cutout.fits [1.4754559, 3.102499, 1.7022967]
DSC00532.cutout.fits [1.7128512, 2.2415109, 4.376782]
DSC00533.cutout.fits [

DSC00667.cutout.fits [2.593698, 3.9893632, 4.84202]
DSC00668.cutout.fits [3.293317, 4.360555, 4.7797623]
DSC00669.cutout.fits [2.5393925, 2.4583836, 4.8814306]
DSC00670.cutout.fits [2.3816183, 2.8319745, 5.0451035]
DSC00671.cutout.fits [2.0508032, 3.9491382, 4.3560214]
DSC00672.cutout.fits [3.6417556, 2.458853, 4.3699408]
DSC00673.cutout.fits [3.1854641, 2.6708488, 4.253669]
DSC00674.cutout.fits [3.0915463, 4.3202925, 4.498146]
DSC00675.cutout.fits [2.6971958, 2.972969, 5.3669987]
DSC00676.cutout.fits [2.5713575, 3.8766372, 5.094847]
DSC00677.cutout.fits [2.4834485, 3.491391, 3.429089]
DSC00678.cutout.fits [2.7663536, 3.6574497, 5.42521]
DSC00679.cutout.fits [2.7838013, 2.478659, 3.2430727]
DSC00680.cutout.fits [2.0504594, 3.252318, 4.55947]
DSC00681.cutout.fits [2.7100065, 3.254584, 3.4799664]
DSC00682.cutout.fits [3.4391687, 2.846894, 3.9442835]
DSC00683.cutout.fits [2.4613109, 2.568311, 5.2763414]
DSC00684.cutout.fits [2.115095, 3.7279902, 4.222732]
DSC00685.cutout.fits [3.9627023, 

DSC00819.cutout.fits [3.0002356, 3.1886833, 4.6856594]
DSC00820.cutout.fits [1.916253, 3.0179312, 2.5708618]
DSC00821.cutout.fits [2.188932, 3.327852, 4.3060117]
DSC00822.cutout.fits [2.0308852, 3.490077, 4.0571303]
DSC00823.cutout.fits [2.7434118, 3.285776, 3.4686632]
DSC00824.cutout.fits [2.063459, 3.6623344, 4.5201087]
DSC00825.cutout.fits [1.7118875, 3.4466157, 4.6531725]
DSC00826.cutout.fits [1.6398152, 3.285582, 2.6505792]
DSC00827.cutout.fits [1.6603086, 3.0131896, 3.1489277]
DSC00828.cutout.fits [2.306682, 3.4893434, 5.082143]
DSC00829.cutout.fits [2.7886548, 3.0071445, 3.0682797]
DSC00830.cutout.fits [2.4237766, 3.0351925, 4.711656]
DSC00831.cutout.fits [2.6425724, 3.6786902, 4.017616]
DSC00832.cutout.fits [2.6394172, 3.684987, 5.6795297]
DSC00833.cutout.fits [1.7959988, 2.26815, 2.7418675]
DSC00834.cutout.fits [2.9267027, 2.6169825, 3.5477836]
DSC00835.cutout.fits [1.4779156, 3.333818, 3.2341125]
DSC00836.cutout.fits [2.315074, 3.1963995, 3.2907035]
DSC00837.cutout.fits [0.91

DSC00971.cutout.fits [2.2830946, 3.382372, 4.0191517]
DSC00972.cutout.fits [1.3272206, 3.6464157, 4.2819853]
DSC00973.cutout.fits [1.5218, 3.368861, 4.2940025]
DSC00974.cutout.fits [1.4145638, 3.8902285, 4.410987]
DSC00975.cutout.fits [2.0318065, 4.35615, 4.3175735]
DSC00976.cutout.fits [2.3230667, 3.8162296, 3.9628704]
DSC00977.cutout.fits [2.5625122, 3.9649317, 3.897204]
DSC00978.cutout.fits [2.1610103, 3.7105043, 4.446582]
DSC00979.cutout.fits [2.9250581, 3.6961935, 4.3197365]
DSC00980.cutout.fits [2.8195813, 3.428164, 3.581186]
DSC00981.cutout.fits [2.429753, 3.7011726, 4.0754094]
DSC00982.cutout.fits [1.810467, 3.5409718, 3.8542314]
DSC00983.cutout.fits [1.8043149, 3.5681973, 4.4522324]
DSC00984.cutout.fits [2.460907, 4.055578, 5.2331944]
DSC00985.cutout.fits [2.8870878, 3.2054229, 4.0263143]
DSC00986.cutout.fits [2.4900737, 4.63228, 3.3188934]
DSC00987.cutout.fits [2.416196, 4.2061424, 3.9762578]
DSC00988.cutout.fits [1.6764356, 3.9581704, 4.4427376]
DSC00989.cutout.fits [1.42103

In [ ]:
coadded_r = output_array[0]
coadded_g = output_array[1]
coadded_b = output_array[2]

In [ ]:
# print(np.mean(coadded_r))
# print(np.mean(coadded_g))
# print(np.mean(coadded_b))

## Subtract background

In case it wasn't subtracted individually from each input image.

In [ ]:
# red_bkg = Background2D(coadded_r, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=bkg_sigma_clip, bkg_estimator=bkg_estimator)
# green_bkg = Background2D(coadded_g, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=bkg_sigma_clip, bkg_estimator=bkg_estimator)
# blue_bkg = Background2D(coadded_b, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=bkg_sigma_clip, bkg_estimator=bkg_estimator)

# coadded_r -= red_bkg.background 
# coadded_g -= green_bkg.background
# coadded_b -= blue_bkg.background 

## Write FITS with result

In [ ]:
# Build 3-extension FITS file with R,G,B drizzled arrays in separate HDUs 
today = datetime.datetime.now().ctime()
drizz_name = os.path.join(datadir, 'drizzle.fits')
print(drizz_name, today)

hdr = fits.Header()
hdr['DATE'] = today
hdr['PATH'] = drizz_name
primary_hdu = fits.PrimaryHDU(header=hdr)

red_hdu   = fits.ImageHDU(coadded_r.astype('float32'))
green_hdu = fits.ImageHDU(coadded_g.astype('float32'))
blue_hdu  = fits.ImageHDU(coadded_b.astype('float32'))

red_hdu.header['BAND'] = 'RED'
green_hdu.header['BAND'] = 'GREEN'
blue_hdu.header['BAND'] = 'BLUE'

hdul = fits.HDUList([primary_hdu, red_hdu, green_hdu, blue_hdu])
hdul.writeto(drizz_name, overwrite=True)

## Plot diagnostics

In [ ]:
hdul = fits.open(drizz_name)
coadded_r = hdul[1].data
coadded_g = hdul[2].data
coadded_b = hdul[3].data

In [ ]:
# aslice = np.index_exp[1100:1290,1900:2100]
# aslice = np.index_exp[800:1800,2200:3200]
# aslice = np.index_exp[1150:1350,2250:2440]
# aslice = np.index_exp[1050:1250,2750:2950] # M31
aslice = np.index_exp[0:coadded_r.shape[0], 0:coadded_r.shape[1]] # Andromeda_2022
# aslice = np.index_exp[700:800,1250:1300] # Andromeda_2022

In [ ]:
vmax = 300

fig = plt.figure(figsize=[12, 12])
ax = fig.add_subplot(2, 2, 1)
plt.imshow(coadded_g[aslice], vmin=0, vmax=vmax)
ax1 = fig.add_subplot(2, 2, 2, sharex=ax, sharey=ax)
plt.imshow(coadded_b[aslice], vmin=0, vmax=vmax)
ax2 = fig.add_subplot(2, 2, 3, sharex=ax, sharey=ax)
plt.imshow(coadded_r[aslice], vmin=0, vmax=vmax)

In [ ]:
fig = plt.figure(figsize=[12, 12])
ax = fig.add_subplot(2, 2, 1)
plt.imshow(output_counts[1][aslice])
plt.colorbar()
ax1 = fig.add_subplot(2, 2, 2, sharex=ax, sharey=ax)
plt.imshow(output_counts[2][aslice])
plt.colorbar()
ax2 = fig.add_subplot(2, 2, 3, sharex=ax, sharey=ax)
plt.imshow(output_counts[0][aslice])
plt.colorbar()

In [ ]:
test_image = coadded_r * 1.93 + coadded_g + coadded_b * 1.90

fig = plt.figure(figsize=[12, 6])
ax = fig.add_subplot(1, 1, 1)
plt.imshow(test_image[aslice], vmin=0, vmax=600)

In [ ]:
rgb_image = make_lupton_rgb(coadded_r * 1.8, coadded_g, coadded_b * 1.8, Q=0.0001, stretch=80., minimum=-10.)

fig = plt.figure(figsize=[12, 6])
ax = fig.add_subplot(1, 1, 1)
plt.imshow(rgb_image[aslice])

In [ ]:
fig = plt.figure(figsize=[12, 6])
ax = fig.add_subplot(1, 1, 1)
plt.imshow(rgb_image)

In [ ]:
rgb_image.shape

In [ ]:
plt.imsave("drizzle.tiff", rgb_image)

In [ ]:
test = coadded_g - coadded_r

vmax = 25

fig = plt.figure(figsize=[12, 12])
ax = fig.add_subplot(1, 1, 1)
plt.imshow(test[aslice], vmin=0, vmax=vmax)